<a href="https://colab.research.google.com/github/zakiarsyad/Vagrantfile/blob/master/PySpark_Recomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 57.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=3881ffd7f034595c4da8157cfa0958fe07606e2e85682fb3dc00ae9e13af947f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.recommendation import ALS

In [ ]:
spark = SparkSession.builder.appName("RS").getOrCreate()

In [ ]:
data = spark.read.csv("movie_ratings.csv", header = True, inferSchema = True)
print(data.printSchema())
print(data.show())

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)

None
+------+------------+------+
|userId|       title|rating|
+------+------------+------+
|   196|Kolya (1996)|     3|
|    63|Kolya (1996)|     3|
|   226|Kolya (1996)|     5|
|   154|Kolya (1996)|     3|
|   306|Kolya (1996)|     5|
|   296|Kolya (1996)|     4|
|    34|Kolya (1996)|     5|
|   271|Kolya (1996)|     4|
|   201|Kolya (1996)|     4|
|   209|Kolya (1996)|     4|
|    35|Kolya (1996)|     2|
|   354|Kolya (1996)|     5|
|   199|Kolya (1996)|     5|
|   113|Kolya (1996)|     2|
|     1|Kolya (1996)|     5|
|   173|Kolya (1996)|     5|
|   360|Kolya (1996)|     4|
|   234|Kolya (1996)|     4|
|    14|Kolya (1996)|     4|
|   309|Kolya (1996)|     4|
+------+------------+------+
only showing top 20 rows

None


In [ ]:
stringIndexer = StringIndexer(inputCol = "title", outputCol = "title_new").fit(data)
data = stringIndexer.transform(data)

In [ ]:
print(data.show(2))

+------+------------+------+---------+
|userId|       title|rating|title_new|
+------+------------+------+---------+
|   196|Kolya (1996)|     3|    287.0|
|    63|Kolya (1996)|     3|    287.0|
+------+------------+------+---------+
only showing top 2 rows

None


In [ ]:
train, test = data.randomSplit([0.7, 0.3])

In [ ]:
rec = ALS(userCol = "userId", itemCol = "title_new", ratingCol = "rating", nonnegative = True).fit(train)

In [ ]:
prediction = rec.transform(test)
print(prediction.show())

+------+--------------------+------+---------+----------+
|userId|               title|rating|title_new|prediction|
+------+--------------------+------+---------+----------+
|     1|   Abyss, The (1989)|     3|    200.0| 3.6760008|
|     1|Ace Ventura: Pet ...|     3|    322.0| 2.7622125|
|     1|      Aladdin (1992)|     4|     95.0| 3.6920524|
|     1|Army of Darkness ...|     4|    289.0| 3.2743216|
|     1|     Bad Boys (1995)|     2|    557.0| 2.6956062|
|     1|Basic Instinct (1...|     3|    327.0| 2.9429388|
|     1|Batman Returns (1...|     1|    219.0| 2.7900822|
|     1| Blade Runner (1982)|     5|     52.0|  4.422504|
|     1|Blues Brothers, T...|     4|     64.0| 4.0312576|
|     1|       Brazil (1985)|     5|    109.0|  4.298029|
|     1|Breakfast at Tiff...|     1|    350.0|   4.08484|
|     1|Brother Minister:...|     4|   1299.0|  3.733507|
|     1| Citizen Ruth (1996)|     4|    641.0| 3.4395516|
|     1|Cyrano de Bergera...|     5|    492.0| 4.1793356|
|     1|D3: Th

In [ ]:
unique_movies = data.select("title_new").distinct()
print(unique_movies.count())

1664


In [ ]:
user_id = 8
watched_movies = data.filter(data["userId"] == user_id).select("title_new").distinct()
print(watched_movies.count())

59


In [ ]:
a = unique_movies.alias("a")
b = watched_movies.alias("b")

In [ ]:
total_movies = a.join(b, a.title_new == b.title_new, how = "left")
print(total_movies.show())

+---------+---------+
|title_new|title_new|
+---------+---------+
|    305.0|     null|
|    596.0|     null|
|    299.0|     null|
|    769.0|     null|
|    692.0|     null|
|    934.0|     null|
|   1051.0|     null|
|    496.0|     null|
|    558.0|     null|
|    170.0|     null|
|    184.0|     null|
|    576.0|     null|
|    147.0|     null|
|    810.0|     null|
|    720.0|     null|
|    782.0|     null|
|   1369.0|     null|
|   1587.0|     null|
|    160.0|     null|
|    608.0|     null|
+---------+---------+
only showing top 20 rows

None


In [ ]:
remaining_movies = total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
print(remaining_movies.show())

+---------+
|title_new|
+---------+
|    305.0|
|    596.0|
|    299.0|
|    769.0|
|    692.0|
|    934.0|
|   1051.0|
|    496.0|
|    558.0|
|    170.0|
|    184.0|
|    576.0|
|    147.0|
|    810.0|
|    720.0|
|    782.0|
|   1369.0|
|   1587.0|
|    160.0|
|    608.0|
+---------+
only showing top 20 rows

None


In [ ]:
remaining_movies = remaining_movies.withColumn("userId", lit(int(user_id)))
print(remaining_movies.show())

+---------+------+
|title_new|userId|
+---------+------+
|    305.0|     8|
|    596.0|     8|
|    299.0|     8|
|    769.0|     8|
|    692.0|     8|
|    934.0|     8|
|   1051.0|     8|
|    496.0|     8|
|    558.0|     8|
|    170.0|     8|
|    184.0|     8|
|    576.0|     8|
|    147.0|     8|
|    810.0|     8|
|    720.0|     8|
|    782.0|     8|
|   1369.0|     8|
|   1587.0|     8|
|    160.0|     8|
|    608.0|     8|
+---------+------+
only showing top 20 rows

None


In [ ]:
prediction = rec.transform(remaining_movies)
prediction = prediction.dropna()
print(prediction.show())

+---------+------+----------+
|title_new|userId|prediction|
+---------+------+----------+
|    305.0|     8| 3.5378315|
|    596.0|     8| 3.3999336|
|    299.0|     8| 3.9661994|
|    769.0|     8| 2.5662215|
|    692.0|     8| 3.0315502|
|    934.0|     8| 1.0555348|
|   1051.0|     8| 3.1642706|
|    496.0|     8| 3.3881295|
|    558.0|     8|  4.177144|
|    170.0|     8| 2.9985428|
|    184.0|     8| 3.1689637|
|    576.0|     8| 3.3694804|
|    147.0|     8|  3.443522|
|    810.0|     8| 2.6838658|
|    720.0|     8| 3.0760283|
|    782.0|     8| 1.5122524|
|   1369.0|     8| 3.3224266|
|   1587.0|     8|   4.39196|
|    160.0|     8| 3.2887082|
|    608.0|     8| 3.0178733|
+---------+------+----------+
only showing top 20 rows

None


In [ ]:
movie_title = IndexToString(inputCol = "title_new", outputCol = "title", labels = stringIndexer.labels)
recommendation = movie_title.transform(prediction)

In [ ]:
print(recommendation.orderBy("prediction", ascending = False).show(3, truncate = False))

+---------+------+----------+----------------------+
|title_new|userId|prediction|title                 |
+---------+------+----------+----------------------+
|1347.0   |8     |5.1891866 |Angel Baby (1995)     |
|1306.0   |8     |4.844149  |Faust (1994)          |
|1198.0   |8     |4.7409034 |Pather Panchali (1955)|
+---------+------+----------+----------------------+
only showing top 3 rows

None
